# Slack の API を叩いてメッセージを取得

In [4]:
# python 3.5.0

import ast
import json
import time
import urllib

In [5]:
class Slack:
    
    # プライベートチャンネルのデータを扱う場合は True
    def __init__(self, token, is_private=False):
        self.token = token
        self.is_private = is_private
        
    # プライベートチャンネルの ID と名前の dict を取得
    def get_channels(self):
        url = "https://slack.com/api/channels.list"
        if self.is_private:
            url = "https://slack.com/api/groups.list"
            
        req = urllib.request.Request(url)
        params = {"token": self.token}
        req.data = urllib.parse.urlencode(params).encode('ascii')
        res = urllib.request.urlopen(req)
        
        if res.status != 200:
            print("error code:", res.status)
            return False
        
        body = json.loads(res.read())

        if self.is_private:    
            return body["groups"]
        else:
            return body["channels"]
    
    
    # プライベートチャンネルのメッセージを取得
    # thread のやり取りは別途 get_threads_replies() で取得
    def get_channel_messages(self, channel_id):
        url = "https://slack.com/api/channels.history"
        if self.is_private:
            url = "https://slack.com/api/groups.history"
            
        req = urllib.request.Request(url)
        # 読み込みの上限は 1000 件
        params = {"token": self.token, "channel": channel_id, "count": 1000, "oldest": 0}
        req.data = urllib.parse.urlencode(params).encode('ascii')
        res = urllib.request.urlopen(req)
        if res.status != 200:
            print("error code:", res.status)
            return False
        
        body = json.loads(res.read())
        messages = body["messages"]
        
        # 1000 件以上ある場合は、1000件目のメッセージより新しいメッセージを
        # 再度1000件ずつ繰り返し読み込む
        while body["has_more"]:
            oldest = body["messages"][-1]["ts"]
            
            req = urllib.request.Request(url)
            params = {"token": self.token, "channel": channel_id, 
                      "count": 1000, "oldest":oldest}
            req.data = urllib.parse.urlencode(params).encode('ascii')
            res = urllib.request.urlopen(req)
            if res.status != 200:
                print("error code:", res.status)
                return False
        
            body = json.loads(res.read())
            
            messages += body["messages"]
            oldest = body["messages"][-1]["ts"]
            
        # messages の dict 
        return messages
    
    
    # private channel 内で thread になったやり取りを取得
    # thread_ts はスレッドになったメッセージには存在
    def get_threads_messages(self, channel_id, thread_ts):
        url = "https://slack.com/api/channels.replies"
        if self.is_private:
            url = "https://slack.com/api/groups.replies"
            
        req = urllib.request.Request(url)
        params = {"token": self.token, "channel": channel_id, "thread_ts": thread_ts}
        req.data = urllib.parse.urlencode(params).encode('ascii')
        res = urllib.request.urlopen(req)
        if res.status != 200:
            print("error code:", res.status)
            return False
        
        body = json.loads(res.read())
        
        return body["messages"]
        
    
# json 形式のデータを良い感じに表示
def print_json(json_data):
    print(json.dumps(json_data, indent=4, separators=(',', ': ')))

In [ ]:
# https://api.slack.com/custom-integrations/legacy-tokens
slack_token = "XXXXXX"

slack = Slack(slack_token)

channel_groups = slack.get_channels()

channel_list = {}
for g in channel_groups:
    channel_list[g["name"]] = g["id"]
    
# general のデータを取得
channel_id = channel_list["general"]
# get_channel_messages で GET できる body の is_limited = True だと、
# free アカウントの制限で全データは取得できない
channel_message = slack.get_channel_messages(channel_id)

with open("./general.json", "w") as f:
    for m in channel_message:
        if "replies" in m and "thread_ts" in m:
            m["replies"] = slack.get_threads_messages(channel_id, m["thread_ts"])
            # スレッドのメッセージを取得する API の Rate limite は 20/minute なので注意
            time.sleep(3.001)
        print(m, file=f)